In [95]:
from pathlib import Path
import importlib
import logging
logging.basicConfig(level=logging.INFO)

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import pandas as pd

from mersearch import helpers
from mersearch.helpers import smiles_to_mol_file_format
from mersearch.vendors import vwr

In [98]:
importlib.reload(helpers)
importlib.reload(vwr)

<module 'mersearch.vendors.vwr' from '/Users/josephkern/Desktop/mersearch/mersearch/vendors/vwr.py'>

In [10]:
profile_path = str(
    Path("~")
    / "Library"
    / "Application Support"
    / "Firefox"
    / "Profiles"
    / "pwofczxw.default"
)
PROXY_HOST = "12.12.12.123"
PROXY_PORT = "1234"
options = Options()
options.set_preference("profile", profile_path)
options.set_preference("network.proxy.type", 1)
options.set_preference("network.proxy.http", PROXY_HOST)
options.set_preference("network.proxy.http_port", int(PROXY_PORT))
options.set_preference("dom.webdriver.enabled", False)
options.set_preference("useAutomationExtension", False)
options.set_preference("excludeSwitches", "enable-automation")
service = Service(r"/usr/local/bin/geckodriver")

configuration = {"service": service, "options": options}
driver = webdriver.Firefox(**configuration)

In [4]:
path = Path.cwd() / '..' / '..' / 'molimg' / 'data' / 'vfs_promising_polymers.csv'
df = pd.read_csv(str(path))

In [5]:
smiles = df.monomer_smiles.to_list()

In [13]:
mol_files = {smile: smiles_to_mol_file_format(smile) for smile in smiles[2:3]}

In [14]:
data = vwr.exact_search(driver, mol_files)

INFO:mersearch.vendors.vwr:Searching for O=C1CCCCCCCCCCCO1 in vwr


In [15]:
data

[{'smiles': 'O=C1CCCCCCCCCCCO1',
  'data': {'property_data': [{'property': 'MWt', 'value': '198.306'},
    {'property': 'MF', 'value': 'C12H22O2'}],
   'name_data': [{'name_type': 'SMILES', 'name': 'O=C1CCCCCCCCCCCO1'},
    {'name_type': 'CAS', 'name': '947-05-7'},
    {'name_type': 'Name', 'name': 'Oxacyclotridecan-2-one'},
    {'name_type': 'Name', 'name': 'QH-1650'}],
   'supplier_data': [{'supplier': 'Combi-Blocks, Inc.',
     'supplier_id': 'QH-1650'}],
   'price_data': [{'supplier': 'Combi-Blocks, Inc.',
     'supplier_id': 'QH-1650',
     'amount': '1',
     'units': 'g',
     'price': '156.00 USD'},
    {'supplier': 'Combi-Blocks, Inc.',
     'supplier_id': 'QH-1650',
     'amount': '10',
     'units': 'g',
     'price': '776.00 USD',
     'unit': 'g'}]}}]

In [17]:
vwr.search_for_mol(driver, smiles_to_mol_file_format('O=C1CCCO1'), 'substructure')

In [68]:
item = WebDriverWait(driver=driver, timeout=30).until(
    EC.presence_of_element_located((By.ID, "hitlist-page-numbers-bottom"))
)

In [100]:
items = item.find_elements(By.TAG_NAME, "a")
item1 = driver.find_element(By.XPATH, '//a[contains(text(),"Next")]')

NoSuchElementException: Message: The element with the reference a1733bda-5304-47c2-a0b5-20c006635e0a is not known in the current browsing context
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.getKnownElement@chrome://remote/content/marionette/element.sys.mjs:471:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:233:33
cloneObject@chrome://remote/content/marionette/json.sys.mjs:56:24
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:244:16
cloneObject@chrome://remote/content/marionette/json.sys.mjs:56:24
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:244:16
json.deserialize@chrome://remote/content/marionette/json.sys.mjs:248:10
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:85:30


In [87]:
for item2 in item1:
    print(item2.text)

Next >>
Next >>


In [89]:
item1.text

'Next >>'

In [90]:
item2.click()

In [99]:
mol_file = smiles_to_mol_file_format(smiles[2])
data = vwr.substructure_search(driver, mol_file)

INFO:mersearch.vendors.vwr:Searching page 1 in vwr


TimeoutException: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:134:16


In [101]:
driver.close()